In [37]:
###
###Imports for the Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib as plt
import os

In [38]:
# get working directory and remove last folder
# TODO: make this more robust
wd = os.path.dirname(os.getcwd())
os.chdir(wd)
print("Working Directory: ", os.getcwd())

Working Directory:  N:\MASTER_DS


In [24]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    "index": np.int32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}


os.chdir("N:\MASTER_DS\Code\Kaggle_competition\Kaggle-seminar\student-performance")
dataset_df = pd.read_csv('data/raw/train.csv', dtype=dtypes, nrows= 2000000)

In [39]:
#preprocessing function to add variables and rescale the time 
def adding_new_variables_rescaling(dataset_df):
    dataset_df = dataset_df.sort_values(['session_id','elapsed_time'])
    dataset_df["elapsed_time"] = dataset_df["elapsed_time"]/1000
    group = dataset_df.groupby(["session_id","level"])["elapsed_time"].diff()
    group = group.fillna(value= 0)
    dataset_df= dataset_df.assign(difference_clicks = group)

    return dataset_df

In [40]:
#preprocessing to combine variables on level stage
#Function to clean the sequential data for the training of the model

#For that Function to work we need to specify the variables in Categorical and Numerical & Counting

CATEGORICAL = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
NUMERICALmean = ['hover_duration',"difference_clicks"]
NUMERICALstd = ['elapsed_time','page','room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration',"difference_clicks"]
COUNTING = ["index"]
MAXIMUM = ["difference_clicks", "elapsed_time"]

def feature_engineer_steve(dataset_df):
    dfs = []
    #add the level group again to make training on the sequential data easier
    tmp = dataset_df.groupby(['session_id','level'])["level_group"].first()
    dfs.append(tmp)
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICALmean:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('mean')
        tmp.name = tmp.name + '_mean'
        dfs.append(tmp)
    for c in NUMERICALstd:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)    
    for c in COUNTING:
        tmp = 1+ dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_sum_of_actions'
        dfs.append(tmp)
    for c in MAXIMUM:
        tmp = dataset_df.groupby(['session_id','level'])[c].agg('max')- dataset_df.groupby(['session_id','level'])[c].agg('min') 
        tmp.name = tmp.name + '_max'
        dfs.append(tmp)
    
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    
     #add Clicks per second afterwards cause we need the time for each level first
    dataset_df["clicks_per_second"] = dataset_df["index_sum_of_actions"]/ dataset_df["elapsed_time_max"]
    
    
    return dataset_df

In [41]:
#return the preprocessed data
##test data preprocessing with Subset of 5 million rows


dataset_df_added = adding_new_variables_rescaling(dataset_df)
dataset_df_level = feature_engineer_steve(dataset_df_added)
print(dataset_df_added)
print(dataset_df_level)

                session_id  index  elapsed_time      event_name       name  \
0        20090312431273200      0         0.000  cutscene_click      basic   
2        20090312431273200      2         0.831    person_click      basic   
3        20090312431273200      3         1.147    person_click      basic   
1        20090312431273200      1         1.323    person_click      basic   
4        20090312431273200      4         1.863    person_click      basic   
...                    ...    ...           ...             ...        ...   
1999995  20110211310637800    472       802.324  navigate_click  undefined   
1999996  20110211310637800    473       802.625  navigate_click  undefined   
1999997  20110211310637800    474       802.900  navigate_click  undefined   
1999998  20110211310637800    475       803.451    object_hover      basic   
1999999  20110211310637800    476       805.500    object_hover  undefined   

         level  page  room_coor_x  room_coor_y  screen_coor_x  

In [28]:
print(dataset_df.groupby(['session_id','level_group'])["level_group"].first())

session_id         level_group
20090312431273200  0-4              0-4
                   13-22          13-22
                   5-12            5-12
20090312433251036  0-4              0-4
                   13-22          13-22
                                  ...  
20110211305198484  13-22          13-22
                   5-12            5-12
20110211310637800  0-4              0-4
                   13-22            NaN
                   5-12            5-12
Name: level_group, Length: 5349, dtype: category
Categories (3, object): ['0-4', '13-22', '5-12']


Here I will build the function to load the training data

we need to throw out some cols of the training dataset before saving


In [31]:
column_names = list(dataset_df_level.columns)
print(column_names)

dtypes={
    "level_group": "category",
    'level': np.uint8,
    'event_name_nunique': np.uint8,
    'name_nunique':np.uint8,
    'fqid_nunique':np.uint8,
    'room_fqid_nunique':np.uint8,
    'text_fqid_nunique':np.uint8, 
    'hover_duration_mean': np.float32,
    'difference_clicks_mean':np.float32,
    'elapsed_time_std':np.float32,
    'page_std':np.float32,
    'room_coor_x_std':np.float32,
    'room_coor_y_std':np.float32,
    'screen_coor_x_std':np.float32,
    'screen_coor_y_std':np.float32,
    'hover_duration_std':np.float32,
    'difference_clicks_std':np.float32,
    'index_sum_of_actions':np.uint16,
    'difference_clicks_max':np.float32,
    'elapsed_time_max':np.float32,
    'clicks_per_second':np.float32
    }

['level', 'level_group', 'event_name_nunique', 'name_nunique', 'fqid_nunique', 'room_fqid_nunique', 'text_fqid_nunique', 'hover_duration_mean', 'difference_clicks_mean', 'elapsed_time_std', 'page_std', 'room_coor_x_std', 'room_coor_y_std', 'screen_coor_x_std', 'screen_coor_y_std', 'hover_duration_std', 'difference_clicks_std', 'index_sum_of_actions', 'difference_clicks_max', 'elapsed_time_max', 'clicks_per_second']


In [32]:
def split_dataset(dataset, test_ratio=0.30):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df_level)
print("{} examples in training, {} examples in testing.".format(
    len(train_x), len(valid_x)))



32761 examples in training, 8193 examples in testing.


In [20]:
# Fetch the unique list of user sessions in the validation dataset. We assigned 
# `session_id` as the index of our feature engineered dataset. Hence fetching 
# the unique values in the index column will give us a list of users in the 
# validation set.
VALID_USER_LIST = valid_x.index.unique()

# Create a dataframe for storing the predictions of each question for all users
# in the validation set.
# For this, the required size of the data frame is: 
# (no: of users in validation set  x no of questions).
# We will initialize all the predicted values in the data frame to zero.
# The dataframe's index column is the user `session_id`s. 
prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)

# Create an empty dictionary to store the models created for each question.
models = {}

# Create an empty dictionary to store the evaluation score for each question.
evaluation_dict ={}

In [ ]:
# Iterate through questions 1 to 18 to train models for each question, evaluate
# the trained model and store the predicted values.
for q_no in range(1,19):

    # Select level group for the question based on the q_no.
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)
    
        
    # Filter the rows in the datasets based on the selected level group. 
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    # Select the labels for the related q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

    # Add the label to the filtered datasets.
    train_df["correct"] = train_labels["correct"]
    valid_df["correct"] = valid_labels["correct"]

    # There's one more step required before we can train the model. 
    # We need to convert the datatset from Pandas format (pd.DataFrame)
    # into TensorFlow Datasets format (tf.data.Dataset).
    # TensorFlow Datasets is a high performance data loading library 
    # which is helpful when training neural networks with accelerators like GPUs and TPUs.
    # We are omitting `level_group`, since it is not needed for training anymore.
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df.loc[:, train_df.columns != 'level_group'], label="correct")
    valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df.loc[:, valid_df.columns != 'level_group'], label="correct")